### `LSTM.py`

In [ ]:
!pip install music21

In [1]:
import glob
import pickle
import numpy
import keras.utils
from music21 import converter, instrument, note, chord, stream, duration
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

2024-04-10 15:07:30.962323: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 15:07:31.007378: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 15:07:31.007433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 15:07:31.008517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 15:07:31.015702: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_files = glob.glob('maestro_v3_2013/*.midi')

In [3]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()
    # notes = pickle.load('/content/drive/My Drive/Colab Notebooks/final_project_neuro240_files/data/notes')

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [4]:
len(train_files)

127

In [5]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in train_files[:90]:
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                # print(str(element.pitch) + "_" + str(element.duration.quarterLength))
                notes.append(str(element.pitch) + "_" + str(element.duration.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
            elif isinstance(element, note.Rest):
                notes.append("R")
    # try:
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    print("Notes saved")
    # except:
    #     pass

    return notes

In [6]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = keras.utils.to_categorical(network_output)

    return (network_input, network_output)

In [7]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [8]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "Rest-Duration-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

In [9]:
train_network()

Parsing maestro_v3_2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_01_R1_2013_wav--2.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_10_13_Group_MID--AUDIO_12_R3_2013_wav--1.midi
Parsing maestro_v3_2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--4.midi
Parsing maestro_v3_2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_02_R1_2013_wav--1.midi
Parsing maestro_v3_2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_17_R3_2013_wav--3.midi
Parsing maestro_v3_2013/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_07_R2_2013_wav--1.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_08_R1_2013_wav--2.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_17_R1_2013_wav--1.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_06_R1_2013_wav--4.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_15_R1_2013_wav--3.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_07_R1_2013_wav--3.midi
Parsing maestro_v3_2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_16_

2024-04-10 15:09:13.871260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 15:09:13.880823: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 15:09:13.883707: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

L355
2024-04-10 15:09:13.890338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 15:09:13.892978: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 15:09:14.080319: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

Epoch 1/200


2024-04-10 15:09:33.988137: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-04-10 15:09:35.040305: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0b00a4b6e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-10 15:09:35.040361: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-10 15:09:35.045965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712761775.118579   21974 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2565/2565 [==============================] - ETA: 0s - loss: 5.0605

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2565/2565 [==============================] - 1057s 408ms/step - loss: 5.0605
Epoch 2/200
2565/2565 [==============================] - 1059s 413ms/step - loss: 4.7363
Epoch 3/200
2565/2565 [==============================] - 1081s 421ms/step - loss: 4.7319
Epoch 4/200
2565/2565 [==============================] - 1089s 424ms/step - loss: 4.7201
Epoch 5/200
2565/2565 [==============================] - 1090s 425ms/step - loss: 4.7181
Epoch 6/200
2565/2565 [==============================] - 1088s 424ms/step - loss: 4.7176
Epoch 7/200
2565/2565 [==============================] - 1086s 424ms/step - loss: 4.7115
Epoch 8/200
2565/2565 [==============================] - 1086s 423ms/step - loss: 4.7090
Epoch 9/200
2565/2565 [==============================] - 1089s 425ms/step - loss: 4.7070
Epoch 10/200
2565/2565 [==============================] - 1091s 425ms/step - loss: 4.6977
Epoch 11/200
2565/2565 [==============================] - 1088s 424ms/step - loss: 4.6877
Epoch 12/200
2565/2565 [=======

### `predict.py`

In [ ]:
WEIGHTS_PATH = "Rest-Duration-weights-improvement-02-4.8267.hdf5"

In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(WEIGHTS_PATH)

    return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                try:
                    new_note = note.Note(current_note.split('_')[0])
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                except:
                    try:
                        new_note = note.Note(current_note)
                        new_note.storedInstrument = instrument.Piano()
                        notes.append(new_note)
                    except:
                        try:
                            new_note = note.Note(int(current_note))
                            new_note.storedInstrument = instrument.Piano()
                            notes.append(new_note)
                        except:
                            pass
            try:
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            except:
                pass
        # pattern is a rest
        elif pattern == "R":
            # print("rest detected")
            new_rest = note.Rest()
            # new_rest.duration.quarterLength = 0.5
            new_rest.storedInstrument = instrument.Piano()
            new_rest.offset = offset
            output_notes.append(new_rest)
        # pattern is a note
        else:
            # print(pattern)
            note_pitch, note_length = pattern.split("_")
            # print((note_pitch, note_length))
            try:
                new_note = note.Note(note_pitch)
                try:
                    new_note.duration = duration.Duration(float(note_length))
                    new_note.storedInstrument = instrument.Piano()
                    new_note.offset = offset
                    output_notes.append(new_note)
                except:
                    new_note.storedInstrument = instrument.Piano()
                    new_note.offset = offset
                    output_notes.append(new_note)
            except:
                pass

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_new_durations.mid')

In [ ]:
generate()